In [1]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}

datos_limpios = pd.read_csv('../data/datos_limpios_filtrados.csv')

In [2]:
datos_limpios = datos_limpios[(datos_limpios['DESCR_CENDIST'] != 'CERVECERA') & (datos_limpios['DESCR_CENDIST'] != 'MODELO')]

In [3]:
datos_limpios = datos_limpios.drop(columns='Unnamed: 0')

In [4]:
datos_limpios

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets
0,ANTOFAGASTA,515,2018-01-02,0.100
1,ANTOFAGASTA,515,2018-01-03,0.125
2,ANTOFAGASTA,515,2018-01-04,0.275
3,ANTOFAGASTA,515,2018-01-05,0.400
4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...
20953083,VALDIVIA,875225,2020-09-25,0.000
20953084,VALDIVIA,875225,2020-09-26,0.000
20953085,VALDIVIA,875225,2020-09-28,0.000
20953086,VALDIVIA,875225,2020-09-29,0.000


In [5]:
def stats(datos):

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * 7, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * 7, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: 1.65 * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

In [7]:
stats_no_agrupados, porcentaje_ss_no_agrupado = stats(datos_limpios)

In [8]:
stats_no_agrupados

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.400533,0.707064,1.765305,1.166655,1.567188,0.744425
1,515,ARICA,0.041857,0.140730,3.362170,0.232205,0.274062,0.847272
2,515,CALAMA,0.219591,0.443948,2.021702,0.732515,0.952106,0.769363
3,515,CASTRO,0.120733,0.300728,2.490851,0.496202,0.616935,0.804302
4,515,CHILLAN,0.276593,0.511619,1.849720,0.844171,1.120763,0.753211
...,...,...,...,...,...,...,...,...
23007,875225,VALDIVIA,0.000280,0.004112,14.661585,0.006785,0.007065,0.960304
23008,875359,SANTIAGO SUR,0.008413,0.242681,28.844410,0.400424,0.408838,0.979421
23009,875365,CURAUMA,0.016827,0.342997,20.383802,0.565945,0.582771,0.971126
23010,875365,SANTIAGO SUR,0.008413,0.242681,28.844410,0.400424,0.408838,0.979421


In [9]:
#stats_no_agrupados[(stats_no_agrupados['MEDIA'] != 0)].to_csv('../data/stats_no_agrupados_limpios.csv')

In [22]:
datos_limpios

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets
0,ANTOFAGASTA,515,2018-01-02,0.100
1,ANTOFAGASTA,515,2018-01-03,0.125
2,ANTOFAGASTA,515,2018-01-04,0.275
3,ANTOFAGASTA,515,2018-01-05,0.400
4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...
34281723,VALDIVIA,875225,2020-09-25,0.000
34281724,VALDIVIA,875225,2020-09-26,0.000
34281725,VALDIVIA,875225,2020-09-28,0.000
34281726,VALDIVIA,875225,2020-09-29,0.000


In [10]:
porcentaje_ss_no_agrupado

0.7668004349888596

In [29]:
datos_para_agrupar = datos_limpios.copy()

In [30]:
FILE_AGRUPACIONES = '../data/agrupacion_sku_70%.xlsx'
sheets_dict = pd.read_excel(FILE_AGRUPACIONES, sheet_name=None)

master = pd.DataFrame()
for name in sheets_dict:
    
    df = sheets_dict[name]
    master = pd.concat([master, df])
    
dict_sku_cd = {}
master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})

for index, row in master.iterrows():
    
    if row['CANTIDAD HUB 3R'] != 0:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1
        
    else:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0

In [31]:
datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 else hubs_dict[x['DESCR_CENDIST']], axis=1)

In [25]:
datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

In [12]:
#datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(lambda x: hubs_dict[x['DESCR_CENDIST']], axis=1)
#datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

In [26]:
stats_agrupados, porcentaje_ss_agrupado = stats(datos_agrupados)

In [14]:
stats_agrupados

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.620125,1.004138,1.619252,1.656828,2.276953,0.727651
1,515,COPIAPO,0.225060,0.463361,2.058834,0.764546,0.989606,0.772576
2,515,COQUIMBO,0.930529,1.629458,1.751110,2.688606,3.619135,0.742886
3,515,COYHAIQUE,0.047326,0.127939,2.703376,0.211100,0.258425,0.816869
4,515,CURAUMA,0.104958,0.271470,2.586467,0.447926,0.552884,0.810163
...,...,...,...,...,...,...,...,...
14377,875225,VALDIVIA,0.002991,0.020249,6.768970,0.033411,0.036402,0.917823
14378,875359,SANTIAGO SUR,0.008413,0.242681,28.844410,0.400424,0.408838,0.979421
14379,875365,CURAUMA,0.016827,0.342997,20.383802,0.565945,0.582771,0.971126
14380,875365,SANTIAGO SUR,0.008413,0.242681,28.844410,0.400424,0.408838,0.979421


In [16]:
porcentaje_ss_agrupado

0.7555242208799188

In [28]:
stats_agrupados[(stats_agrupados['MEDIA'] != 0)].to_csv('../data/stats_agrupados_limpios_70%.csv')

In [18]:
stats_agrupados['STOCK_TEORICO'].sum() - stats_no_agrupados['STOCK_TEORICO'].sum()

-2843.9188887511846